# Redes Convolucionales

En este ejercicio de práctica vamos a implementar las capas convolucionales y de pooling vistas en la teoría, utilizando Numpy. Si bien, ambas capas ya vienen implementadas en las librerías que utilizaremos luego en el resto del curso, resulta interesante poder programarlas desde cero para afianzar el entendimiento de como funcionan en su interior.

#### Notación

Antes de empezar definimos la notación utilizada cuando trabajamos con este tipo de capas en una red neuronal:

- $n_H$, $n_W$ y $n_C$ son la altura, el ancho y número de canales de una determinada capa, respectivamente.

- El superíndice $[l]$ denota el número de capa: 
    - Ejemplo: $a^{[3]}$, $W^{[3]}$ y $b^{[3]}$ son la activación, los pesos y los biases de la $3^{ra}$ capa, respectivamente.

- El superíndice $(i)$ denota el número de ejemplo utilizado en el entrenamiento:
    - Ejemplo: $x^{(i)}$ es el $i$-ésimo ejemplo de entrada.
    
- El subíndice $i$ denota el número de elemento en un vector:
    - Ejemplo: $a^{[l]}_i$ es la $i$-ésima activación en la capa $l$ (si asumimos una capa Fully Connected).

---

### Importación de paquetes
 

In [ ]:
import numpy as np
import torch

## Capa Convolucional

Para implementar una capa convolucional y su forward pass vamos a dividir el trabajo en 3 funciones separadas. En primer lugar, escribiremos una función que nos permita realizar padding con ceros en la cantidad que nosotros deseemos. Luego, haremos una función para realizar el computo de la operación de convolución en sí, es decir, una función que nos retorne el resultado de aplicar la convolución de un filtro determinado en una posición fija de la matriz de entrada. Por último, utilizaremos las dos funciones previas para implementar la operación de convolución completa que realizaria una capa de una red neuronal, sobre un volumen de entrada, con un número determinado de filtros convolucionales.

---

### Padding con ceros

El padding con ceros, o Zero-padding, consiste en agregar ceros en los bordes de la matriz de entrada, lo cual tiene dos razones principales:

- Evitar que el ancho y alto de los volumenes dentro de una red neuronal convolucional profunda disminuyan, lo cual limitaría la profundidad que puede tener la red. 
- Utilizar mejor la información que se encuentra en los bordes de las matrices de entrada en una capa convolucional. Sin la existencia de este padding, pocos valores de la salida dependen de la información en estos bordes.

Para implementar facilmente esta operación aprovechamos la funcion [`pad`](https://numpy.org/doc/stable/reference/generated/numpy.pad.html) de Numpy.

In [ ]:
# Función para padding con ceros

def zero_padding(X, pad):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (batch_size, n_C, n_H, n_W)
    pad: Entero representando la cantidad de filas y columnas que se deben agregar con ceros

  Retorna:
    X_padded: Array numpy con dimensiones (batch_size, n_C, n_H + 2*pad, n_W + 2*pad)
  """

  X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))

  return X_padded

### Operación de convolución

En esta segunda función implementaremos la operación de convolución utilizada dentro de las capas convolucionales. Nuestra función, entonces, tomara un recorte del volumen que entra en la capa convolucional, cuyo tamaño es igual al del filtro de pesos, lo convolucionará con dicho filtro (multiplicando elemento a elemento y sumando todos los resultados) y le sumará el bias.

Para relizar la suma de todos los elementos en un array de Numpy utilizaremos la función [`sum`](httpshttps://numpy.org/doc/stable/reference/generated/numpy.sum.html://)

In [ ]:
# Función para realizar la operación de convolución

def convolve(X, W, b):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
    W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
    b: Entero con el valor de bias de la capa actual

  Retorna:
    Z: Entero con el valor del resultado
  """

  # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
  aux = X * W
  # Realizo la suma de todos los elementos
  aux = np.sum(aux)
  # Le sumo el valor del bias para obtener Z
  Z = aux + float(b)

  return Z

### Forward Pass - Capa Convolucional

Para realizar el forward pass de una capa convolucional debemos tomar varios filtros y convolucionarlos a lo largo y ancho de todo el volumen de entrada a la capa. El resultado de esta operación será una matriz de dos dimensiones por cada uno de los filtros que contenga la capa, las cuales tenemos que apilar para conformar el volumen de salida.

<center>
<img width="620" height="440" src="https://drive.google.com/uc?id=1H5dI5IlRPktyyIX79uio_dTa7MLTE1oE">
</center>

Al implementar esta función debemos tener en cuenta todos los hiperparametros que tienen las capas convolucionales, los cuales influyen tanto en las porciones de la matriz de entrada que tomamos para convolucionar con los filtros de la capa, como así también en las dimensiones del volumen de salida mediante las siguientes formulas:

$$ n^{[l]}_H = \lfloor \frac{n^{[l-1]}_H - f + 2 \times p}{s} \rfloor +1 $$
$$ n^{[l]}_W = \lfloor \frac{n^{[l-1]}_W - f + 2 \times p}{s} \rfloor +1 $$
$$ n^{[l]}_C = \text{Cantidad de filtros de la $l$-ésima capa}$$

Comenzamos agregando el padding correspondiente al volumen de entrada. Luego, para cada ejemplo en el batch, seleccionamos ventanas de dicho volumen, respetando los valores de `stride` y `filter_size`, sobre las cuales computaremos la operación de convolución con cada uno de los filtros que compongan la capa. Para saber donde comienza y termina cada una de estas ventanas vamos a utilizar variables internas (`y_start` y `y_end` para el eje vertical, y `x_start` y `x_end` para el horizontal) calculadas a partir de iterar sobre las dimensiones del volumen de salida.

In [ ]:
# Función para realizar el forward pass de una capa convolucional

def conv_forward(layer_input, W, b, stride, padding):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
    W: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
    b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
    stride: Entero con el valor de stride utilizado en la capa actual.
    padding: 

  Retorna:
    Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Obtengo las dimensiones de los filtros
  (n_C, n_C_prev, filter_size, filter_size) = W.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  n_H = int((n_H_prev + 2*padding - filter_size)/stride + 1)
  n_W = int((n_W_prev + 2*padding - filter_size)/stride + 1)

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Agrego padding con ceros al volumen de entrada
  layer_input_padded = zero_padding(layer_input, padding)

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        x_start = stride * w
        x_end = x_start + filter_size

        # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
        slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el valor del filtro y bias del canal correspondiente
          filter = W[c, :, :, :]
          bias = b[c]

          # Computo la operación de convolución para esta ventana
          Z[i, c, h, w] = convolve(slice_from_input_padded, filter, bias)
  
  return Z

In [ ]:
# Dimensiones de la entrada
batch_size = 10
input_height, input_width = (7, 7)
input_channels = 4

# Dimensiones de la convolucional
filters = 8
filter_size = 3
stride = 2
pad = 1

# Variables de prueba
test_array = np.random.randn(batch_size, input_channels, input_height, input_width)
W = np.random.randn(filters, input_channels, filter_size, filter_size)
b = np.random.randn(filters)


conv_result = conv_forward(test_array, W, b, stride, pad)
conv_result_pyt = torch.nn.functional.conv2d(torch.tensor(test_array), torch.tensor(W), torch.tensor(b), stride, pad)

assert(conv_result.shape == conv_result_pyt.shape)
print("Convolución: Result shape: {}".format(conv_result.shape))
print("Convolución: Result value: {}".format(conv_result[1, 1, 1, 1]))
print("Convolución: Result shape: {}".format(conv_result_pyt.shape))
print("Convolución: Result value: {}".format(conv_result_pyt[1, 1, 1, 1]))


## Capa Pooling

La capa de pooling realiza una operación más simple que la capa convolucional. En este caso, simplemente se reducen las dimensiones verticales y horizontales del volumen de entrada, sin afectar su profundidad (ya que no existen los filtros). Dado que las capas de pooling pueden ser de tipo "Max-Pooling" o "Average-Pooling", implementaremos a continuación, una función para cada tipo de operación.

![Example-of-max-pooling-and-average-pooling-operations-In-this-example-a-4x4-image-is.png](https://drive.google.com/uc?id=1QjV72N9yAlgxwAyzRKOzjjH6rqkHfTFB)

In [ ]:
def max_pooling_forward(layer_input, filter_size, stride):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa max-pooling (batch_size, n_C_prev, n_H_prev, n_W_prev)
    filter_size: Entero con el valor de tamaño de filtro utilizado en la capa actual. 
    stride: Entero con el valor de stride utilizado en la capa actual.

  Retorna:
    Z: Array numpy con los valores de salida de la capa max-pooling (batch_size, n_C, n_H, n_W)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  n_H = int(1 + (n_H_prev - filter_size) / stride)
  n_W = int(1 + (n_W_prev - filter_size) / stride)
  n_C = n_C_prev

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana
        x_start = stride * w
        x_end = x_start + filter_size
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el maximo
          Z[i, c, h, w] = np.max(layer_input[i, c, y_start:y_end, x_start:x_end])

  return Z

In [ ]:
def average_pooling_forward(layer_input, filter_size, stride):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa max-pooling (batch_size, n_C_prev, n_H_prev, n_W_prev)
    filter_size: Entero con el valor de tamaño de filtro utilizado en la capa actual. 
    stride: Entero con el valor de stride utilizado en la capa actual.

  Retorna:
    Z: Array numpy con los valores de salida de la capa average-pooling (batch_size, n_C, n_H, n_W)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  n_H = int(1 + (n_H_prev - filter_size) / stride)
  n_W = int(1 + (n_W_prev - filter_size) / stride)
  n_C = n_C_prev

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana
        x_start = stride * w
        x_end = x_start + filter_size
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el maximo
          Z[i, c, h, w] = np.mean(layer_input[i, c, y_start:y_end, x_start:x_end])

  return Z

In [ ]:
# Dimensiones de la entrada
batch_size = 10
input_height, input_width = (10, 10)
input_channels = 3

np.random.seed(1)

test_array = np.random.randn(batch_size, input_channels, input_height, input_width)
stride = 2
filter_size = 2

max_result = max_pooling_forward(test_array, filter_size, stride)
max_result_pyt = torch.nn.functional.max_pool2d(torch.tensor(test_array), filter_size, stride)

assert(max_result.shape == max_result_pyt.shape)
print("Max-Pooling: Result shape: {}".format(max_result.shape))
print("Max-Pooling: Result value: {}".format(max_result[0, 0, 0, 0]))
print("Max-Pooling: Result shape: {}".format(max_result_pyt.shape))
print("Max-Pooling: Result value: {}".format(max_result_pyt[0, 0, 0, 0]))

average_result = average_pooling_forward(test_array, filter_size, stride)
average_result_pyt = torch.nn.functional.avg_pool2d(torch.tensor(test_array), filter_size, stride)

assert(max_result.shape == average_result_pyt.shape)
print("Average-Pooling: Result shape: {}".format(average_result.shape))
print("Average-Pooling: Result value: {}".format(average_result[0, 0, 0, 0]))
print("Average-Pooling: Result shape: {}".format(average_result_pyt.shape))
print("Average-Pooling: Result value: {}".format(average_result_pyt[0, 0, 0, 0]))